In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import re
import os
from scripts.ari import ari_to_features
import argparse
import logging

from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

logger = logging.getLogger(__name__)

/Users/anadwi/Documents/Projects/B_Ragi/B_Ragi/venv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None


def show_album_tracks(album):
    tracks = []
    results = sp.album_tracks(album['id'])
    tracks.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    for i, track in enumerate(tracks):
        logger.info('%s. %s', i+1, track['name'])
    
    return tracks
        
    


def show_artist_albums(artist):
    albums = []
    trks = []
    results = sp.artist_albums(artist['id'], album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    logger.info('Total albums: %s', len(albums))
    unique = set()  # skip duplicate albums
    for album in albums:
        name = album['name'].lower()
        if name not in unique:
            logger.info('ALBUM: %s', name)
            unique.add(name)
            trks.extend(show_album_tracks(album))
    
    return trks


def show_artist(artist):
    logger.info('====%s====', artist['name'])
    logger.info('Popularity: %s', artist['popularity'])
    if len(artist['genres']) > 0:
        logger.info('Genres: %s', ','.join(artist['genres']))
        

def get_all_tracks(artist):
    
    art_d = get_artist(artist)
    tracks_df = pd.DataFrame(show_artist_albums(art_d))
    return tracks_df

In [3]:
df = pd.read_csv('../data/song_features.csv')

In [4]:
# instead of reading directly on the artists column, I processed it to extract the relevant name and Id information.
from ast import literal_eval
df['artist_name'] = df['artists'].apply(lambda x: literal_eval(x)[0]['name'].lower())
df['artist_id'] = df['artists'].apply(lambda x: literal_eval(x)[0]['id'])

df['song_label'] = df['artist_name'].apply(lambda x: 1 if x in ("hiroshi yoshimura","haruomi hosono","hosono haruomi","inoyamaland") else 0)

In [6]:
df.columns

Index(['Unnamed: 0', 'artists', 'available_markets', 'disc_number',
       'duration_ms_x', 'explicit', 'external_urls', 'href', 'id', 'is_local',
       'name', 'preview_url', 'track_number', 'type_x', 'uri_x',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type_y', 'uri_y', 'track_href', 'analysis_url', 'duration_ms_y',
       'time_signature', 'artist_pop', 'genres', 'track_pop', 'artist_name',
       'artist_id', 'song_label'],
      dtype='object')

In [55]:
import os
# add api info
client_id = 'e5d27af30b04420baec4921a610d3b4e'
secret = '830c1e45c2644274a06c163782ee54f7'
redirect = 'http://127.0.0.1:8080'
os.environ['SPOTIPY_CLIENT_ID'] = 'e5d27af30b04420baec4921a610d3b4e'
os.environ['SPOTIPY_CLIENT_SECRET'] = '830c1e45c2644274a06c163782ee54f7'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://127.0.0.1:8080'

In [56]:
df.artist_name.unique()

array(['hiroshi yoshimura', 'inoyamaland', 'endel', 'green-house',
       'brian eno', 'robert fripp', 'toto', 'passengers', 'roger eno',
       'tom rogerson', 'eno • hyde', 'harmonia', 'david byrne',
       'jah wobble', 'james', 'harold budd', 'laraaji', 'jon hassell',
       'cluster', 'haruomi hosono', 'hosono haruomi', 'ノンスタンダード合唱団',
       '中原香織', 'masataka matsutoya', 'takahiko ishikawa',
       'harry hosono & the world shyness', 'shigeru suzuki',
       'tatsuro yamashita', 'emily a. sprague', 'lightbath', 'vangelis',
       'irina valentinova', 'jon & vangelis', 'kritikes madares',
       'psarantonis', 'loudovicos ton anogion', 'irene papas',
       'vassilis saleas', 'jon anderson', 'mariangela celeste'],
      dtype=object)

In [13]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [57]:
#"inoyamaland": "Wasser",
song_dict = {
    #artist, song
    "hiroshi yoshimura": "Green",
    "green-house":"Chysis",
    "brian eno":"The Plateaux of Mirror",
    "inoyamaland":"Wasser",
    "haruomi hosono":"The endless talking",
    "hosono haruomi": "The endless talking",
    'emily a. sprague':"Horizon",
    "vangelis":"Blade Runner Blues"
}

In [15]:
filt_df = df[df["artist_name"].isin(song_dict.keys())]
filt_df.artist_name.unique()

array(['hiroshi yoshimura', 'inoyamaland', 'green-house', 'brian eno',
       'haruomi hosono', 'hosono haruomi', 'emily a. sprague', 'vangelis'],
      dtype=object)

In [16]:
songs_df = filt_df[filt_df['name'].isin(song_dict.values())]

In [17]:
filt_df.valence.describe()

count    1407.000000
mean        0.269723
std         0.283332
min         0.009090
25%         0.039500
50%         0.128000
75%         0.448500
max         0.983000
Name: valence, dtype: float64

In [12]:
songs_df[songs_df['artist_name'] == "hiroshi yoshimura"]['artist_id']

4    1DGpHnPOpMYY780hcQHmPB
Name: artist_id, dtype: object

In [13]:
songs_df[songs_df['artist_name'] == "inoyamaland"]['artist_id']

104    3nYCvyP4RxuKyEKygqxWHy
Name: artist_id, dtype: object

In [14]:
df[df['artist_name'] == 'green-house']['artist_id']

129    0M6QGBKWICr8dxhh3UJW45
130    0M6QGBKWICr8dxhh3UJW45
131    0M6QGBKWICr8dxhh3UJW45
132    0M6QGBKWICr8dxhh3UJW45
133    0M6QGBKWICr8dxhh3UJW45
134    0M6QGBKWICr8dxhh3UJW45
135    0M6QGBKWICr8dxhh3UJW45
136    0M6QGBKWICr8dxhh3UJW45
137    0M6QGBKWICr8dxhh3UJW45
Name: artist_id, dtype: object

In [15]:
songs_df['id'].to_list()

['0HEzGwip1s4seBM0s4fQhW',
 '30SZ15hpz4Fgu59FpOybhx',
 '28GQO7qa54ouDsWDjKtl0n',
 '575blCgesVtCu0HEYaIcas']

In [18]:
sp.recommendation_genre_seeds()

{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',


In [58]:
# recommend songs using hiroshi yoshimura, greenhouse with min_valence = 0.44
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
# seed_artists = ["1DGpHnPOpMYY780hcQHmPB","3nYCvyP4RxuKyEKygqxWH","0M6QGBKWICr8dxhh3UJW45"]
seed_artsts = ["1DGpHnPOpMYY780hcQHmPB","0M6QGBKWICr8dxhh3UJW45"]
seed_gnres = ["ambient"]
#seed_trcks = songs_df['id'].to_list()

results = sp.recommendations(seed_artists=seed_artsts,
                             seed_genres=seed_gnres,
                             min_valence=0.44)

In [29]:
rec_df = pd.DataFrame(results['tracks'])
rec_df['artist_name'] = rec_df['artists'].apply(lambda x: literal_eval(str(x))[0]['name'].lower())
rec_df['artist_id'] = rec_df['artists'].apply(lambda x: literal_eval(str(x))[0]['id'])

In [37]:
af = []
for tk_id in rec_df['id']:
    af.append(sp.audio_features(tk_id)[0])

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.589,0.363000,2,-11.905,1,0.0283,0.876,0.6760,0.6240,0.879,101.609,audio_features,1vAdz9eeL6jVQ20WRJsm4L,spotify:track:1vAdz9eeL6jVQ20WRJsm4L,https://api.spotify.com/v1/tracks/1vAdz9eeL6jV...,https://api.spotify.com/v1/audio-analysis/1vAd...,67547,4
1,0.581,0.555000,1,-10.129,1,0.0353,0.763,0.1840,0.0902,0.641,90.443,audio_features,5HRfgCymXuFWpdkhpyvxAr,spotify:track:5HRfgCymXuFWpdkhpyvxAr,https://api.spotify.com/v1/tracks/5HRfgCymXuFW...,https://api.spotify.com/v1/audio-analysis/5HRf...,213999,3
2,0.733,0.326000,5,-14.195,0,0.0351,0.855,0.9030,0.1070,0.590,140.273,audio_features,1XON3mjzv0wGZgsrzxr5cO,spotify:track:1XON3mjzv0wGZgsrzxr5cO,https://api.spotify.com/v1/tracks/1XON3mjzv0wG...,https://api.spotify.com/v1/audio-analysis/1XON...,166500,4
3,0.490,0.296000,6,-18.490,1,0.0584,0.969,0.9410,0.1020,0.548,110.566,audio_features,1m54vFgkAORsMpRTy55QUN,spotify:track:1m54vFgkAORsMpRTy55QUN,https://api.spotify.com/v1/tracks/1m54vFgkAORs...,https://api.spotify.com/v1/audio-analysis/1m54...,234055,4
4,0.665,0.361000,11,-19.317,1,0.0469,0.991,0.9010,0.1040,0.591,96.338,audio_features,6Xcnbv35WPrlYPJ0TjbOF1,spotify:track:6Xcnbv35WPrlYPJ0TjbOF1,https://api.spotify.com/v1/tracks/6Xcnbv35WPrl...,https://api.spotify.com/v1/audio-analysis/6Xcn...,201253,3
5,0.379,0.180000,4,-14.504,1,0.0415,0.994,0.8820,0.1100,0.856,100.831,audio_features,0DvdODtKwuo4XPTeDOSlm4,spotify:track:0DvdODtKwuo4XPTeDOSlm4,https://api.spotify.com/v1/tracks/0DvdODtKwuo4...,https://api.spotify.com/v1/audio-analysis/0Dvd...,139587,4
6,0.627,0.321000,10,-18.837,0,0.0521,0.813,0.9140,0.1160,0.784,169.046,audio_features,6VI6LnXQSm7NI7ptuQ7Dl4,spotify:track:6VI6LnXQSm7NI7ptuQ7Dl4,https://api.spotify.com/v1/tracks/6VI6LnXQSm7N...,https://api.spotify.com/v1/audio-analysis/6VI6...,304145,4
7,0.222,0.216000,5,-18.692,0,0.0439,0.993,0.9520,0.1620,0.580,72.794,audio_features,3y7lez26VNRGyrnjEuBnsD,spotify:track:3y7lez26VNRGyrnjEuBnsD,https://api.spotify.com/v1/tracks/3y7lez26VNRG...,https://api.spotify.com/v1/audio-analysis/3y7l...,151693,5
8,0.820,0.702000,7,-10.176,1,0.0460,0.846,0.8670,0.1750,0.786,106.595,audio_features,0zNnrjAohPi8ghYD3t5HIX,spotify:track:0zNnrjAohPi8ghYD3t5HIX,https://api.spotify.com/v1/tracks/0zNnrjAohPi8...,https://api.spotify.com/v1/audio-analysis/0zNn...,259693,4
9,0.833,0.231000,7,-11.424,0,0.1640,0.936,0.8560,0.1010,0.921,101.018,audio_features,5FNR7DtBHTdLCLsDF0y3kH,spotify:track:5FNR7DtBHTdLCLsDF0y3kH,https://api.spotify.com/v1/tracks/5FNR7DtBHTdL...,https://api.spotify.com/v1/audio-analysis/5FNR...,195000,4


In [39]:
audio_features_df = pd.DataFrame(af)
rec_af = rec_df.merge(audio_features_df, on='id')
audio_features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.589,0.363000,2,-11.905,1,0.0283,0.876,0.6760,0.6240,0.879,101.609,audio_features,1vAdz9eeL6jVQ20WRJsm4L,spotify:track:1vAdz9eeL6jVQ20WRJsm4L,https://api.spotify.com/v1/tracks/1vAdz9eeL6jV...,https://api.spotify.com/v1/audio-analysis/1vAd...,67547,4
1,0.581,0.555000,1,-10.129,1,0.0353,0.763,0.1840,0.0902,0.641,90.443,audio_features,5HRfgCymXuFWpdkhpyvxAr,spotify:track:5HRfgCymXuFWpdkhpyvxAr,https://api.spotify.com/v1/tracks/5HRfgCymXuFW...,https://api.spotify.com/v1/audio-analysis/5HRf...,213999,3
2,0.733,0.326000,5,-14.195,0,0.0351,0.855,0.9030,0.1070,0.590,140.273,audio_features,1XON3mjzv0wGZgsrzxr5cO,spotify:track:1XON3mjzv0wGZgsrzxr5cO,https://api.spotify.com/v1/tracks/1XON3mjzv0wG...,https://api.spotify.com/v1/audio-analysis/1XON...,166500,4
3,0.490,0.296000,6,-18.490,1,0.0584,0.969,0.9410,0.1020,0.548,110.566,audio_features,1m54vFgkAORsMpRTy55QUN,spotify:track:1m54vFgkAORsMpRTy55QUN,https://api.spotify.com/v1/tracks/1m54vFgkAORs...,https://api.spotify.com/v1/audio-analysis/1m54...,234055,4
4,0.665,0.361000,11,-19.317,1,0.0469,0.991,0.9010,0.1040,0.591,96.338,audio_features,6Xcnbv35WPrlYPJ0TjbOF1,spotify:track:6Xcnbv35WPrlYPJ0TjbOF1,https://api.spotify.com/v1/tracks/6Xcnbv35WPrl...,https://api.spotify.com/v1/audio-analysis/6Xcn...,201253,3
5,0.379,0.180000,4,-14.504,1,0.0415,0.994,0.8820,0.1100,0.856,100.831,audio_features,0DvdODtKwuo4XPTeDOSlm4,spotify:track:0DvdODtKwuo4XPTeDOSlm4,https://api.spotify.com/v1/tracks/0DvdODtKwuo4...,https://api.spotify.com/v1/audio-analysis/0Dvd...,139587,4
6,0.627,0.321000,10,-18.837,0,0.0521,0.813,0.9140,0.1160,0.784,169.046,audio_features,6VI6LnXQSm7NI7ptuQ7Dl4,spotify:track:6VI6LnXQSm7NI7ptuQ7Dl4,https://api.spotify.com/v1/tracks/6VI6LnXQSm7N...,https://api.spotify.com/v1/audio-analysis/6VI6...,304145,4
7,0.222,0.216000,5,-18.692,0,0.0439,0.993,0.9520,0.1620,0.580,72.794,audio_features,3y7lez26VNRGyrnjEuBnsD,spotify:track:3y7lez26VNRGyrnjEuBnsD,https://api.spotify.com/v1/tracks/3y7lez26VNRG...,https://api.spotify.com/v1/audio-analysis/3y7l...,151693,5
8,0.820,0.702000,7,-10.176,1,0.0460,0.846,0.8670,0.1750,0.786,106.595,audio_features,0zNnrjAohPi8ghYD3t5HIX,spotify:track:0zNnrjAohPi8ghYD3t5HIX,https://api.spotify.com/v1/tracks/0zNnrjAohPi8...,https://api.spotify.com/v1/audio-analysis/0zNn...,259693,4
9,0.833,0.231000,7,-11.424,0,0.1640,0.936,0.8560,0.1010,0.921,101.018,audio_features,5FNR7DtBHTdLCLsDF0y3kH,spotify:track:5FNR7DtBHTdLCLsDF0y3kH,https://api.spotify.com/v1/tracks/5FNR7DtBHTdL...,https://api.spotify.com/v1/audio-analysis/5FNR...,195000,4


In [40]:
rec_df['artist_name'].unique()

array(['yasuaki shimizu', 'susumu yokota', 'sam gendel',
       'satoshi & makoto', 'steve reich', 'alabaster deplume',
       'ana roxanne', 'carlos niño & friends', 'domenique dumont',
       'masakatsu takagi', 'raymond scott', 'hiroshi yoshimura', 'meitei',
       'satoshi ashikawa', 'inoyamaland', '越智義朗', 'colloboh'],
      dtype=object)

In [41]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
# seed_artists = ["1DGpHnPOpMYY780hcQHmPB","3nYCvyP4RxuKyEKygqxWH","0M6QGBKWICr8dxhh3UJW45"]
seed_artsts = ["1DGpHnPOpMYY780hcQHmPB","0M6QGBKWICr8dxhh3UJW45"]
seed_gnres = ["ambient"]
#seed_trcks = songs_df['id'].to_list()

results = sp.recommendations(seed_artists=seed_artsts,seed_genres=seed_gnres, min_valence=0.13)

In [42]:
rec_df2 = pd.DataFrame(results['tracks'])
rec_df2['artist_name'] = rec_df2['artists'].apply(lambda x: literal_eval(str(x))[0]['name'].lower())
rec_df2['artist_id'] = rec_df2['artists'].apply(lambda x: literal_eval(str(x))[0]['id'])

In [43]:
rec_df2['artist_name'].unique()

array(['roedelius', 'green-house', 'gigi masin', 'louie zong',
       'omni gardens', 'ana roxanne', 'sofie birch', 'hiroshi yoshimura',
       'satoshi & makoto', 'salamanda', 'inoyamaland',
       'the sweet enoughs', 'mary lattimore', 'colloboh', 'jacober',
       'khotin', 'alabaster deplume', 'rachika nayar'], dtype=object)

In [46]:
f_list = []
for i in tqdm([uri for uri in rec_df2['id'].unique()]):
    try:
        f_list.append(sp.audio_features(i)[0])
    except:
        continue

a_f2 = pd.DataFrame(f_list)
a_f2.plot(y='tempo',figsize=(16,6))

100%|██████████████████████████████████████████████| 20/20 [00:02<00:00,  7.96it/s]


ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

# Artist Details

* name: Michi
* artist_uri: 0Cw7Bk8wlbj3HhzzQ6Tnl9
* album_id : 42BnqrIhngLi3r7Lo04rOe

https://open.spotify.com/album/42BnqrIhngLi3r7Lo04rOe?si=beQk_xdqRiG_jdXEXwdiwg

In [152]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

album = {}
album['id'] = '42BnqrIhngLi3r7Lo04rOe'
mc_df = pd.DataFrame(show_album_tracks(album))

In [153]:
featureLIST = []

featureLIST = []

for i in tqdm([uri for uri in mc_df['id'].unique()]):
    try:
        featureLIST.append(ari_to_features(i))
    except:
        continue
        
mc_f_df = pd.DataFrame(featureLIST)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.50it/s]


In [164]:
col_int = ['duration_ms_x', 'id', 'name', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type_y', 'uri_y',
       'track_href', 'analysis_url', 'duration_ms_y', 'time_signature',
       'artist_pop', 'genres', 'track_pop']

In [166]:
mc_f_t = mc_df.merge(mc_f_df, on='id',how='left')
mc_f_t[col_int].T

,0,1,2,3,4
duration_ms_x,241266,171666,256346,342893,318880
id,508sfGdc6MTU42OctfkeB5,6TM3KAMu0ImERiFB9Rbkw5,12ByIXvlKwlhQ7DtrGQUsH,2Fa9eEBImzaNeY02GRbdnx,7m1NungNg0qHMnY7WvoiWQ
name,Field Trips,Coronado,Heartbroken Dolphin,Rodney The Cosmic Entity,Belvedere
danceability,0.0727,0.425,0.257,0.481,0.225
energy,0.306,0.305,0.377,0.643,0.386
key,6,5,9,0,0
loudness,-15.048,-10.635,-9.508,-9.636,-8.752
mode,1,1,0,1,1
speechiness,0.0379,0.0293,0.0308,0.034,0.0462
acousticness,0.928,0.976,0.785,0.951,0.0779


# Recommendation Experiments

seed_limit : 5
min_valuence: (min_songs, mean ?)


## Strategy 1
Generate playlist using the artist as an inspiration with other artists

In [24]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
# seed_artists = ["1DGpHnPOpMYY780hcQHmPB","3nYCvyP4RxuKyEKygqxWH","0M6QGBKWICr8dxhh3UJW45"]
seed_artsts = ["0Cw7Bk8wlbj3HhzzQ6Tnl9","1DGpHnPOpMYY780hcQHmPB","0M6QGBKWICr8dxhh3UJW45"]
seed_gnres = ["ambient"]
#seed_trcks = songs_df['id'].to_list()

results = sp.recommendations(seed_artists=seed_artsts,seed_genres=seed_gnres, min_valence=0.13)

In [25]:
rec_df2 = pd.DataFrame(results['tracks'])
rec_df2['artist_name'] = rec_df2['artists'].apply(lambda x: literal_eval(str(x))[0]['name'].lower())
rec_df2['artist_id'] = rec_df2['artists'].apply(lambda x: literal_eval(str(x))[0]['id'])

In [26]:
rec_df2.shape

(20, 19)

In [27]:
rec_df2.columns

Index(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms',
       'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local',
       'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri',
       'artist_name', 'artist_id'],
      dtype='object')

In [173]:
from spotipy.oauth2 import SpotifyOAuth
scope = 'playlist-modify-public'
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
user_id = sp.me()['id']
sp.user_playlist_create(user_id, "ambient_recc")

{'collaborative': False,
 'description': None,
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/3l86yGS5qGRFrwgvasDMev'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/3l86yGS5qGRFrwgvasDMev',
 'id': '3l86yGS5qGRFrwgvasDMev',
 'images': [],
 'name': 'ambient_recc',
 'owner': {'display_name': 'anand_dw',
  'external_urls': {'spotify': 'https://open.spotify.com/user/anand_dw'},
  'href': 'https://api.spotify.com/v1/users/anand_dw',
  'id': 'anand_dw',
  'type': 'user',
  'uri': 'spotify:user:anand_dw'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSw1NDBjN2MyNWM4OWNjN2VlYTg2ZTdlOTg2Njk1NGY2NjM1YjNjNmRk',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/3l86yGS5qGRFrwgvasDMev/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:3l86yGS5qGRFrwgvasDMev'}

{'collaborative': False,
 'description': None,
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/3l86yGS5qGRFrwgvasDMev'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/3l86yGS5qGRFrwgvasDMev',
 'id': '3l86yGS5qGRFrwgvasDMev',
 'images': [],
 'name': 'ambient_recc',
 'owner': {'display_name': 'anand_dw',
  'external_urls': {'spotify': 'https://open.spotify.com/user/anand_dw'},
  'href': 'https://api.spotify.com/v1/users/anand_dw',
  'id': 'anand_dw',
  'type': 'user',
  'uri': 'spotify:user:anand_dw'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSw1NDBjN2MyNWM4OWNjN2VlYTg2ZTdlOTg2Njk1NGY2NjM1YjNjNmRk',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/3l86yGS5qGRFrwgvasDMev/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:3l86yGS5qGRFrwgvasDMev'}

In [177]:
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
user_id = sp.me()['id']
sp.playlist_add_items('3l86yGS5qGRFrwgvasDMev', rec_df2['id'].to_list())

{'snapshot_id': 'MixkOTdhZjBhZjM4YzFmMmYzMDNkMjUzZTZhNTAxMjRhOWM3MWU3OTYx'}